# surfrad database interface

This database contains information about things that do not change all that frequently:

- instrument deployment
- instrument inventory

QC related databases, which change quit frequently,0 are disconnected to tis database as long as we are not having a central server that prevents simultanius writing.

In [1]:
import surfradpy.database as srfdb

In [12]:
reload(srfdb)

<module 'surfradpy.database' from '/Users/htelg/prog/SURFRAD/surfradpy/database.py'>

In [13]:
p2db = 'surfrad_database.db'
db = srfdb.SurfradDatabase(p2db)

In [51]:
db.snapshot()['tables']

['sites',
 'instrument_type',
 'channels_wl',
 'clearsky_fits',
 'sqlite_sequence',
 'deployments',
 'instruments_mfrsr']

In [11]:
db.dump_table('deployments')

,index,Date_start,Date_stop,Location,instrument_type_id,instrument_type,instrument_sn,facing
0,0,2014-06-23 00:00:00,2014-08-20 00:00:00,BAO,1,mfrsr,453,unk
1,1,2014-09-15 00:00:00,nan,SKA,1,mfrsr,453,unk
2,2,2016-04-22 00:00:00,2018-10-15 00:00:00,MLO,1,mfrsr,453,unk
3,3,2018-10-16 00:00:00,nan,CUCF,1,mfrsr,453,unk
4,4,2020-03-12 00:00:00,2020-03-24 00:00:00,SKA?,1,mfrsr,453,unk
...,...,...,...,...,...,...,...,...
273,4,2021-08-01 00:00:00,nan,YES,1,mfrsr,688,unk
274,0,2020-12-15 00:00:00,nan,2D401,1,mfrsr,689,unk
275,1,2021-03-01 00:00:00,2021-03-15 00:00:00,CUCF,1,mfrsr,689,unk
276,3,2021-10-01 00:00:00,planned,Kettle-Ponds,1,mfrsr,689,unk


## some useful commands

In [37]:
# delete table
db.execute_query('DROP TABLE IF EXISTS [what to delet];')

# update sites table

In [52]:
import atmPy.data_archives.NOAA_ESRL_GMD_GRAD.surfrad.surfrad as atmsrf

In [184]:
overwrite = True

In [190]:
df = db.dump_table('sites')
df

,index,site_id,abb,name,elevation,latitude,longitude,network
0,0,1,tbl,Table Mountain,1689,40.12498,-105.23680,surfrad
1,1,2,bnd,Bondville,230,40.05192,-88.37309,surfrad
2,2,3,sxf,Sioux Falls,473,43.73403,-96.62328,surfrad
3,3,4,dra,Desert Rock,1007,36.62373,-116.01947,surfrad
4,4,5,fpk,Fort Peck,634,48.30783,-105.10170,surfrad
5,5,6,gwn,Goodwin Creek,98,34.25470,-89.87290,surfrad
6,6,7,psu,Penn. State Univ.,376,40.72012,-77.93085,surfrad


In [186]:
# df = df.sort_values('site_id')

In [187]:
for site in atmsrf._locations:
    abb = site['abbreviation'][0].lower()
    if (df.abb == abb).sum() > 0:
        if not overwrite:
            print(f'{abb} already in table ... skip')
            continue
        else:
            exists = df[df.abb == abb].iloc[0]
            site_id = exists.site_id
            index = exists.name
            df = df.drop(index)
    else:
        site_id  = df.site_id.max() + 1
        index = df.index.max() + 1
        
    
    row = dict(site_id = site_id,
               abb = abb,
               name = site['name'],
               network = 'surfrad',
               elevation = site['alt'],
               latitude = site['lat'],
               longitude = site['lon'],
              )
    row
         
    
    df = pd.concat([df,pd.DataFrame(row, index=[index])])

df = df.sort_values('site_id')

In [188]:
df

,site_id,abb,name,elevation,latitude,longitude,network
0,1,tbl,Table Mountain,1689,40.12498,-105.23680,surfrad
1,2,bnd,Bondville,230,40.05192,-88.37309,surfrad
2,3,sxf,Sioux Falls,473,43.73403,-96.62328,surfrad
3,4,dra,Desert Rock,1007,36.62373,-116.01947,surfrad
4,5,fpk,Fort Peck,634,48.30783,-105.10170,surfrad
5,6,gwn,Goodwin Creek,98,34.25470,-89.87290,surfrad
6,7,psu,Penn. State Univ.,376,40.72012,-77.93085,surfrad


In [189]:
if 0:
    with sqlite3.connect(db.path2db) as dbt:
        df.to_sql('sites', dbt, if_exists='replace')

# update MFRSR_History

In [16]:
out = db.update_mfrsr_history('MFRSR_History.xlsx', 
                              replace=True,
                              # dryrun=True
                             )

Instrument 451 has fist column with the name Purchased SN 451 w/stand from eBay July 2024 not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 607 has fist column with the name Brookhaven MFRSR not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 559 has fist column with the name Thermopile MFRSR (Standard filters) not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?
Instrument 584 has fist column with the name MFRSR purchased around 2013 not "Date_start" as expected. This indicates that the instrument was never put into action ... bought for spare parts?


In [2]:
db

NameError: name 'db' is not defined